In [1]:
import os
import sys

sys.path.insert(0, '..')

## Setup the model area

In [2]:
from morpheus.modflow.types.discretization.spatial.GridCells import GridCells
from morpheus.modflow.types.discretization import SpatialDiscretization
from morpheus.modflow.types.discretization.spatial import Rotation, LengthUnit, Grid, Crs
from morpheus.modflow.types.geometry import Polygon, LineString

polygon = Polygon(
  type='Polygon',
  coordinates=[[
    (13.922514437551428, 50.964720483303836),
    (13.925250781947113, 50.965228748412386),
    (13.925036413951403, 50.96623732041704),
    (13.92222441026388, 50.96629040370362),
    (13.922514437551428, 50.964720483303836)
  ]]
)

rotation = Rotation(0)
length_unit = LengthUnit.meters()
x_coordinates = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
y_coordinates = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

grid = Grid.from_polygon_with_relative_coordinates(
  polygon=polygon,
  x_coordinates=x_coordinates,
  y_coordinates=y_coordinates,
  rotation=rotation,
)

spatial_discretization = SpatialDiscretization(
  geometry=polygon,
  grid=grid,
  affected_cells=GridCells.from_polygon(polygon=polygon, grid=grid),
  crs=Crs.from_str('EPSG:4326')
)

spatial_discretization



SpatialDiscretization(geometry=Polygon(coordinates=[[(13.922514437551428, 50.964720483303836), (13.925250781947113, 50.965228748412386), (13.925036413951403, 50.96623732041704), (13.92222441026388, 50.96629040370362), (13.922514437551428, 50.964720483303836)]], type='Polygon'), grid=Grid(origin=Point(coordinates=(13.922224410263878, 50.9662904037036), type='Point'), x_distances=[33.68941547288559, 33.68941547288559, 33.6894154728856, 33.689415472885585, 33.6894154728856, 33.6894154728856, 33.68941547288557, 33.6894154728856, 33.68941547288563, 33.68941547288557], y_distances=[27.74944856585935, 27.74944856585935, 27.749448565859346, 27.749448565859353, 27.74944856585934, 27.749448565859353, 27.749448565859325, 27.74944856585938, 27.749448565859325, 27.749448565859353], rotation=Rotation(value=0), length_unit=LengthUnit(unit=2)), affected_cells=GridCells(shape=(10, 10), data=[GridCell(x=0, y=0, value=True), GridCell(x=0, y=1, value=True), GridCell(x=0, y=2, value=True), GridCell(x=0, y=

## Setup Time Discretization

In [3]:
from morpheus.modflow.types.discretization.time.TimeUnit import TimeUnit
from morpheus.modflow.types.discretization.time.Stressperiods import StartDateTime, StressPeriodCollection, \
  StressPeriod, NumberOfTimeSteps, TimeStepMultiplier, IsSteadyState
from morpheus.modflow.types.discretization.time import TimeDiscretization
from datetime import datetime

time_discretization = TimeDiscretization(
  start_date_time=StartDateTime.from_datetime(datetime(2020, 1, 1)),
  end_date_time=StartDateTime.from_datetime(datetime(2020, 12, 31)),
  stress_periods=StressPeriodCollection([
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 1, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.yes()
    ),
  ]),
  time_unit=TimeUnit.days()
)

time_discretization

TimeDiscretization(start_date_time=StartDateTime(value=datetime.datetime(2020, 1, 1, 0, 0)), end_date_time=StartDateTime(value=datetime.datetime(2020, 12, 31, 0, 0)), stress_periods=StressPeriodCollection(values=[StressPeriod(start_date_time=StartDateTime(value=datetime.datetime(2020, 1, 1, 0, 0)), number_of_time_steps=NumberOfTimeSteps(value=1), time_step_multiplier=TimeStepMultiplier(value=1), steady_state=IsSteadyState(value=True))]), time_unit=TimeUnit(unit=4))

# Setup Modflow Model

In [4]:
from morpheus.modflow.types.ModflowModel import ModflowModel

modflow_model = ModflowModel.new()
modflow_model = modflow_model.with_updated_time_discretization(time_discretization)
modflow_model = modflow_model.with_updated_spatial_discretization(spatial_discretization=spatial_discretization)

In [5]:
import folium

m = folium.Map(tiles="cartodbpositron", crs="EPSG3857", zoom_start=12, location=[50.965, 13.922])
m.add_child(folium.GeoJson(polygon.as_geojson()))

m.fit_bounds(m.get_bounds())
m

# Setup constant head boundary with 1 observation

In [6]:
from morpheus.modflow.types.geometry import Point
from morpheus.modflow.types.boundaries.Boundary import BoundaryName, ObservationId
from morpheus.modflow.types.soil_model import LayerId
from morpheus.modflow.types.boundaries.ConstantHead import ConstantHead, ConstantHeadObservation, DataItem, StartHead, \
  EndHead

layers_list = [LayerId.new()]
constant_head = ConstantHead.from_geometry(
  name=BoundaryName('constant_head'),
  geometry=LineString([
    (13.9223, 50.9662),
    (13.9250, 50.96615),
  ]),
  grid=grid,
  affected_layers=[modflow_model.soil_model.layers[0].id],
  observations=[
    ConstantHeadObservation(
      id=ObservationId.new(),
      geometry=Point((13.9223, 50.9662)),
      raw_data=[
        DataItem(
          date_time=StartDateTime.from_datetime(datetime(2019, 1, 1)),
          start_head=StartHead.from_float(100),
          end_head=EndHead.from_float(100),
        ),
        DataItem(
          date_time=StartDateTime.from_datetime(datetime(2021, 12, 31)),
          start_head=StartHead.from_float(120),
          end_head=EndHead.from_float(120),
        ),
      ]
    )
  ]
)

m.add_child(folium.GeoJson(constant_head.as_geojson()))
m.add_child(folium.GeoJson(constant_head.affected_cells.as_geojson(grid).as_geojson()))
for constant_head.observation in constant_head.observations:
  m.add_child(folium.GeoJson(constant_head.observation.as_geojson()))
m.fit_bounds(m.get_bounds())
m

In [7]:
from morpheus.modflow.types.boundaries.Boundary import BoundaryCollection

modflow_model = modflow_model.with_updated_boundaries(boundaries=BoundaryCollection.from_list([constant_head]))

## Calculate

In [8]:
from morpheus.modflow.infrastructure.calculation.modflow_2005.types.Calculation import Modflow2005Calculation
from morpheus.modflow.infrastructure.calculation.modflow_2005.types.CalculationProfile import Mf2005CalculationProfile

calculation_profile = Mf2005CalculationProfile.new()
mf2005_calculation = Modflow2005Calculation.new(modflow_model=modflow_model, calculation_profile=calculation_profile)
mf2005_calculation

Modflow2005Calculation(calculation_id=CalculationId(value='1ffc4891-d387-4247-bcbe-589bf104bccc'), modflow_model=ModflowModel(model_id=ModelId(value='167e6be8-4c83-456d-875e-6729ca8aff45'), spatial_discretization=SpatialDiscretization(geometry=Polygon(coordinates=[[(13.922514437551428, 50.964720483303836), (13.925250781947113, 50.965228748412386), (13.925036413951403, 50.96623732041704), (13.92222441026388, 50.96629040370362), (13.922514437551428, 50.964720483303836)]], type='Polygon'), grid=Grid(origin=Point(coordinates=(13.922224410263878, 50.9662904037036), type='Point'), x_distances=[33.68941547288559, 33.68941547288559, 33.6894154728856, 33.689415472885585, 33.6894154728856, 33.6894154728856, 33.68941547288557, 33.6894154728856, 33.68941547288563, 33.68941547288557], y_distances=[27.74944856585935, 27.74944856585935, 27.749448565859346, 27.749448565859353, 27.74944856585934, 27.749448565859353, 27.749448565859325, 27.74944856585938, 27.749448565859325, 27.749448565859353], rotatio

In [9]:
import os
mf2005_calculation.preprocess(
  model_ws=os.path.join(os.getcwd(), 'data', mf2005_calculation.calculation_id.to_str()),
  exe_name=os.path.join(os.getcwd(), 'bin', 'mf2005'),
)
mf2005_calculation.write_input()

In [10]:
mf2005_calculation.run()


FloPy is using the following executable to run the model: ../../bin/mf2005

                                  MODFLOW-2005     
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL
                             Version 1.12.00 2/3/2017                        

 Using NAME file: 167e6be8-4c83-456d-875e-6729ca8aff45.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2023/12/05 19:56:09

 Solving:  Stress period:     1    Time step:     1    Ground-Water Flow Eqn.
 Run end date and time (yyyy/mm/dd hh:mm:ss): 2023/12/05 19:56:09
 Elapsed run time:  0.003 Seconds

  Normal termination of simulation
